In [58]:
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import math
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from statsmodels.tsa.stattools import adfuller
from keras.models import Sequential
from keras.layers import SimpleRNN, Dense, Dropout
from keras.optimizers import SGD
import warnings
warnings.filterwarnings('ignore')

In [59]:
df = pd.read_excel('Data c&w.xlsx')

In [60]:
df.tail()

,Year,DJF,JFM,FMA,MAM,AMJ,MJJ,JJA,JAS,ASO,SON,OND,NDJ
70,2020,0.5,0.5,0.4,0.2,-0.1,-0.3,-0.4,-0.6,-0.9,-1.2,-1.3,-1.2
71,2021,-1.0,-0.9,-0.8,-0.7,-0.5,-0.4,-0.4,-0.5,-0.7,-0.8,-1.0,-1.0
72,2022,-1.0,-0.9,-1.0,-1.1,-1.0,-0.9,-0.8,-0.9,-1.0,-1.0,-0.9,-0.8
73,2023,-0.7,-0.4,-0.1,0.2,0.5,0.8,1.1,1.3,1.6,1.8,1.9,2.0
74,2024,1.8,1.5,1.1,0.7,0.4,0.2,0.0,-0.1,NaN,NaN,NaN,NaN


In [61]:
df.dropna(inplace=True)

In [62]:
df

,Year,DJF,JFM,FMA,MAM,AMJ,MJJ,JJA,JAS,ASO,SON,OND,NDJ
0,1950,-1.5,-1.3,-1.2,-1.2,-1.1,-0.9,-0.5,-0.4,-0.4,-0.4,-0.6,-0.8
1,1951,-0.8,-0.5,-0.2,0.2,0.4,0.6,0.7,0.9,1.0,1.2,1.0,0.8
2,1952,0.5,0.4,0.3,0.3,0.2,0.0,-0.1,0.0,0.2,0.1,0.0,0.1
3,1953,0.4,0.6,0.6,0.7,0.8,0.8,0.7,0.7,0.8,0.8,0.8,0.8
4,1954,0.8,0.5,0.0,-0.4,-0.5,-0.5,-0.6,-0.8,-0.9,-0.8,-0.7,-0.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,2019,0.7,0.7,0.7,0.7,0.5,0.5,0.3,0.1,0.2,0.3,0.5,0.5
70,2020,0.5,0.5,0.4,0.2,-0.1,-0.3,-0.4,-0.6,-0.9,-1.2,-1.3,-1.2
71,2021,-1.0,-0.9,-0.8,-0.7,-0.5,-0.4,-0.4,-0.5,-0.7,-0.8,-1.0,-1.0
72,2022,-1.0,-0.9,-1.0,-1.1,-1.0,-0.9,-0.8,-0.9,-1.0,-1.0,-0.9,-0.8


In [63]:
#### Replace all column names by overwritting on it

cols = ['Year','January','February','March','April','May','June','July','August','September','October','November','December']

df.columns = cols

df.head(10)

,Year,January,February,March,April,May,June,July,August,September,October,November,December
0,1950,-1.5,-1.3,-1.2,-1.2,-1.1,-0.9,-0.5,-0.4,-0.4,-0.4,-0.6,-0.8
1,1951,-0.8,-0.5,-0.2,0.2,0.4,0.6,0.7,0.9,1.0,1.2,1.0,0.8
2,1952,0.5,0.4,0.3,0.3,0.2,0.0,-0.1,0.0,0.2,0.1,0.0,0.1
3,1953,0.4,0.6,0.6,0.7,0.8,0.8,0.7,0.7,0.8,0.8,0.8,0.8
4,1954,0.8,0.5,0.0,-0.4,-0.5,-0.5,-0.6,-0.8,-0.9,-0.8,-0.7,-0.7
5,1955,-0.7,-0.6,-0.7,-0.8,-0.8,-0.7,-0.7,-0.7,-1.1,-1.4,-1.7,-1.5
6,1956,-1.1,-0.8,-0.6,-0.5,-0.5,-0.5,-0.6,-0.6,-0.5,-0.4,-0.4,-0.4
7,1957,-0.2,0.1,0.4,0.7,0.9,1.1,1.3,1.3,1.3,1.4,1.5,1.7
8,1958,1.8,1.7,1.3,0.9,0.7,0.6,0.6,0.4,0.4,0.4,0.5,0.6
9,1959,0.6,0.6,0.5,0.3,0.2,-0.1,-0.2,-0.3,-0.1,0.0,0.0,0.0


In [64]:
# Mengonversi kolom tanggal menjadi objek tanggal (jika perlu)
if 'Date' in df.columns:
   df['Date'] = pd.to_datetime(df['Year'], errors='coerce')

# Tetapkan kolom tanggal sebagai indeks jika ada
if 'Date' in df.columns:
    df.set_index('Date', inplace=True)

# Visualisasikan baris pertama DataFrame untuk memahami strukturnya
print("Baris pertama DataFrame:")
print(df.head(51))

Baris pertama DataFrame:
    Year  January  February  March  April  May  June  July  August  September  \
0   1950     -1.5      -1.3   -1.2   -1.2 -1.1  -0.9  -0.5    -0.4       -0.4   
1   1951     -0.8      -0.5   -0.2    0.2  0.4   0.6   0.7     0.9        1.0   
2   1952      0.5       0.4    0.3    0.3  0.2   0.0  -0.1     0.0        0.2   
3   1953      0.4       0.6    0.6    0.7  0.8   0.8   0.7     0.7        0.8   
4   1954      0.8       0.5    0.0   -0.4 -0.5  -0.5  -0.6    -0.8       -0.9   
5   1955     -0.7      -0.6   -0.7   -0.8 -0.8  -0.7  -0.7    -0.7       -1.1   
6   1956     -1.1      -0.8   -0.6   -0.5 -0.5  -0.5  -0.6    -0.6       -0.5   
7   1957     -0.2       0.1    0.4    0.7  0.9   1.1   1.3     1.3        1.3   
8   1958      1.8       1.7    1.3    0.9  0.7   0.6   0.6     0.4        0.4   
9   1959      0.6       0.6    0.5    0.3  0.2  -0.1  -0.2    -0.3       -0.1   
10  1960     -0.1      -0.1   -0.1    0.0  0.0   0.0   0.1     0.2        0.3   
11 

In [65]:
# Setting 80 percent data for training
training_data_len = math.ceil(len(df) * .8)
training_data_len 

#Splitting the dataset
train_data = df[:training_data_len].iloc[:,:1] 
test_data = df[training_data_len:].iloc[:,:1]
print(train_data.shape, test_data.shape)

(60, 1) (14, 1)


In [66]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 74 entries, 0 to 73
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Year       74 non-null     int64  
 1   January    74 non-null     float64
 2   February   74 non-null     float64
 3   March      74 non-null     float64
 4   April      74 non-null     float64
 5   May        74 non-null     float64
 6   June       74 non-null     float64
 7   July       74 non-null     float64
 8   August     74 non-null     float64
 9   September  74 non-null     float64
 10  October    74 non-null     float64
 11  November   74 non-null     float64
 12  December   74 non-null     float64
dtypes: float64(12), int64(1)
memory usage: 8.1 KB


In [67]:
# Selecting Month values
dataset_train = train_data.Year.values 
# Reshaping 1D to 2D array
dataset_train = np.reshape(dataset_train, (-1,1)) 
dataset_train.shape

(60, 1)

In [68]:
scaler = MinMaxScaler(feature_range=(0,1))
# scaling dataset
scaled_train = scaler.fit_transform(dataset_train)

print(scaled_train[:5])

[[0.        ]
 [0.01694915]
 [0.03389831]
 [0.05084746]
 [0.06779661]]


In [69]:
# Selecting Open Price values
dataset_test = test_data.Year.values 
# Reshaping 1D to 2D array
dataset_test = np.reshape(dataset_test, (-1,1))  
# Normalizing values between 0 and 1
scaled_test = scaler.fit_transform(dataset_test)  
print(*scaled_test[:5])

[0.] [0.07692308] [0.15384615] [0.23076923] [0.30769231]


In [70]:
X_train = []
y_train = []
for i in range(50, len(scaled_train)):
    X_train.append(scaled_train[i-50:i, 0])
    y_train.append(scaled_train[i, 0])
    if i <= 51:
        print(X_train)
        print(y_train)
        print()

[array([0.        , 0.01694915, 0.03389831, 0.05084746, 0.06779661,
       0.08474576, 0.10169492, 0.11864407, 0.13559322, 0.15254237,
       0.16949153, 0.18644068, 0.20338983, 0.22033898, 0.23728814,
       0.25423729, 0.27118644, 0.28813559, 0.30508475, 0.3220339 ,
       0.33898305, 0.3559322 , 0.37288136, 0.38983051, 0.40677966,
       0.42372881, 0.44067797, 0.45762712, 0.47457627, 0.49152542,
       0.50847458, 0.52542373, 0.54237288, 0.55932203, 0.57627119,
       0.59322034, 0.61016949, 0.62711864, 0.6440678 , 0.66101695,
       0.6779661 , 0.69491525, 0.71186441, 0.72881356, 0.74576271,
       0.76271186, 0.77966102, 0.79661017, 0.81355932, 0.83050847])]
[0.8474576271186436]

[array([0.        , 0.01694915, 0.03389831, 0.05084746, 0.06779661,
       0.08474576, 0.10169492, 0.11864407, 0.13559322, 0.15254237,
       0.16949153, 0.18644068, 0.20338983, 0.22033898, 0.23728814,
       0.25423729, 0.27118644, 0.28813559, 0.30508475, 0.3220339 ,
       0.33898305, 0.3559322 , 0.372

In [71]:
X_test = []
y_test = []
for i in range(50, len(scaled_test)):
    X_test.append(scaled_test[i-50:i, 0])
    y_test.append(scaled_test[i, 0])

In [72]:
# The data is converted to Numpy array
X_train, y_train = np.array(X_train), np.array(y_train)

#Reshaping
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1],1))
y_train = np.reshape(y_train, (y_train.shape[0],1))
print("X_train :",X_train.shape,"y_train :",y_train.shape)

X_train : (10, 50, 1) y_train : (10, 1)


In [73]:
# The data is converted to numpy array
X_test, y_test = np.array(X_test), np.array(y_test)

# Jika X_test hanya 1D, tambahkan dimensi kedua
if len(X_test.shape) == 1:
    X_test = np.reshape(X_test, (X_test.shape[0], 1, 1))  # Misalnya 1 fitur per sampel
elif len(X_test.shape) == 2:
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))  # Tambahkan dimensi ketiga

# Reshape y_test hanya jika diperlukan (misalnya 2D ke 1D)
y_test = np.reshape(y_test, (y_test.shape[0], 1))

print("X_test:", X_test.shape, "y_test:", y_test.shape)

X_test: (0, 1, 1) y_test: (0, 1)


In [74]:
# initializing the RNN
regressor = Sequential()

# adding RNN layers and dropout regularization
regressor.add(SimpleRNN(units = 50, 
                        activation = "tanh",
                        return_sequences = True,
                        input_shape = (X_train.shape[1],1)))
regressor.add(Dropout(0.2))

regressor.add(SimpleRNN(units = 50, 
                        activation = "tanh",
                        return_sequences = True))

regressor.add(SimpleRNN(units = 50,
                        activation = "tanh",
                        return_sequences = True))

regressor.add( SimpleRNN(units = 50))

# adding the output layer
regressor.add(Dense(units = 1,activation='sigmoid'))

# compiling RNN
regressor.compile(optimizer = SGD(learning_rate=0.01,
                                  decay=1e-6, 
                                  momentum=0.9, 
                                  nesterov=True), 
                  loss = "mean_squared_error")

# fitting the model
regressor.fit(X_train, y_train, epochs = 20, batch_size = 2)
regressor.summary()

Epoch 1/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 9s 26ms/step - loss: 0.1486
Epoch 2/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0050
Epoch 3/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.0036
Epoch 4/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 0.0018  
Epoch 5/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 0.0024
Epoch 6/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 0.0027
Epoch 7/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 0.0029
Epoch 8/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 0.0022
Epoch 9/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 0.0021
Epoch 10/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - loss: 0.0026
Epoch 11/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 53ms/step - loss: 0.0025
Epoch 12/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - loss: 0.0038
Epoch 13/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 0.0019
Epoch 14/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 0.0029
Epoch 15/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 0.0014
Epoch 16/20
5/5 ━━━━━━━━━━━━━━━━

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ simple_rnn_8 (SimpleRNN)        │ (None, 50, 50)         │         2,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 50, 50)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_9 (SimpleRNN)        │ (None, 50, 50)         │         5,050 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_10 (SimpleRNN)       │ (None, 50, 50)         │         5,050 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_11 (SimpleRNN)       │ (None, 50)             │         5,050 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 35,604 (139.08 KB)

 Trainable params: 17,801 (69.54 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 17,803 (69.55 KB)

In [75]:
# predictions with X_test data
y_RNN = regressor.predict(X_test)

ValueError: math domain error